In [ ]:
!pip install waveplate

In [ ]:
# BB84 protocol -- QKD -- with 2 qubit
from waveplate import * 

qc = PhtotonicCircuit()

qc.BS(135) # beam splitter

# data can be any 2 bit string
data = '01'

alice_basis = 'XX' # 2 basis for 2 qubits
bob_basis = 'XZ' # it can be Z or X

# data encoding with half wave plates
if data[0] == '1': qc.HWP(135,0) # turn qubit 0 to the |RV> state with phase of e^(iπ)
if data[1] == '1': qc.HWP(135,1) # turn the qubit 1 to the |DV> state with phase of e^(iπ)

# apply basis
def apply_basis(basis1):
  for idx,i in enumerate(basis1):
    if i == 'X':
      qc.HWP(22.5,idx)
    if i == 'Z':
      qc.HWP(45,idx)
apply_basis(alice_basis)
apply_basis(bob_basis)


qc.mirror(135) # apply mirror to all paths

qc.PBS(90) # polarizing beam splitter

probs = qc.measure_probs() # get the probalilities

print(probs)

bit0 = {'|DH>':0,'|RV>':1}
bit1 = {'|DV>':1,'|RH>':0}


st = list('00')
error = {0:0,1:0}
for i in probs.keys():
  if len(probs) == 2:
    if i in bit0: st[0] = str(bit0[i])
    if i in bit1: st[1] = str(bit1[i])
  else:
    if i in bit0: error[0] += 1
    if i in bit1: error[1] += 1

st = ''.join(st)
if st == data: print('got the same data')
else:
  for idx,i in enumerate(error.keys()):
    if error[i] > 1: print(f'bit {idx} is wrong looks lke alice and bob didnt apply the same basis cancel it out')
    else: print(f'bit {idx} matches alices :)')

"""
This is the diagram if data = '01'
                       alice_basis = 'XX'
                       bob_basis = 'XZ'

>----BS----HWP(22.5)--HWP(22.5)---\
     |                          |
  HWP(135)                      |
     |                         PBS--- photoresisitor1
  HWP(22.5)                      |
     |                   photoresisitor2
   HWP(45)
     |
     \----------PBS---- photoresisitor3
                 |
           photoresisitor4
         """
